# Text generation using Amazon Titan and Model Parameters
There are model parameters which can be tuned to get variance(deterministic/non-deterministic) of response based on your needs.
Most common ones in text generation models are, 
- temperature
- topP
- maxTokenCount
- stopSequence
This example demonstrates how you can use this with Amazon Titan while generating a text - in this case a poem

## Step 1: Configure AWS environment

In [1]:
# NOT RECOMMENDED FOR PRODUCTION DEPLOYMENT OR FOR SHARING IN PUBLIC.
import os
# os.environ['AWS_ACCESS_KEY_ID'] = 'Your key id'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'Your key'

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

## Step 2: Explore and choose the model of your choice
We will be using `amazon.titan-text-express-v1` in this example

In [3]:
# import boto3 to access AWS resources using python
import boto3

# bedrock client to explore the available foundation models
bedrock = boto3.client(service_name='bedrock',region_name='us-east-1')

In [4]:
# To list available models
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'ddbe5769-08e4-45b7-8767-f3c589372214',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 07 May 2025 04:26:47 GMT',
   'content-type': 'application/json',
   'content-length': '54644',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ddbe5769-08e4-45b7-8767-f3c589372214'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
   'modelId': 'amazon.titan-image-generator-v1:0',
   'modelName': 'Titan Image Generator G1',
   'providerName': 'Amazon',

In [5]:
# To get information about the model
bedrock.get_foundation_model(modelIdentifier='amazon.titan-text-express-v1')

{'ResponseMetadata': {'RequestId': '991f6049-e46b-4b8e-9f82-04dead83d0d2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 07 May 2025 04:27:23 GMT',
   'content-type': 'application/json',
   'content-length': '524',
   'connection': 'keep-alive',
   'x-amzn-requestid': '991f6049-e46b-4b8e-9f82-04dead83d0d2'},
  'RetryAttempts': 0},
 'modelDetails': {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-text-express-v1',
  'modelId': 'amazon.titan-text-express-v1',
  'modelName': 'Titan Text G1 - Express',
  'providerName': 'Amazon',
  'inputModalities': ['TEXT'],
  'outputModalities': ['TEXT'],
  'responseStreamingSupported': True,
  'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND'],
  'modelLifecycle': {'status': 'ACTIVE'}}}

## Step 3: Prepare prompt for the model¶
Prompt format and model parameters can be found on the relevant model documentaion page. Example - https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-text.html
```
{
    "inputText": string,
    "textGenerationConfig": {
        "temperature": float,  
        "topP": float,
        "maxTokenCount": int,
        "stopSequences": [string]
    }
}
```

In [6]:
# Lets use json module to construct the prompt
import json

req_with_model_params = json.dumps({'inputText': 'Write a poem about sky',
                                         'textGenerationConfig':{
                                              'temperature': 1.0,
                                               'topP': 1.0,
                                               'maxTokenCount': 50
                                         }})
print(req_with_model_params)

{"inputText": "Write a poem about sky", "textGenerationConfig": {"temperature": 1.0, "topP": 1.0, "maxTokenCount": 50}}


## Step 4: Create bedrock runtime to invoke the model

In [7]:
bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

## Step 5: Invoke the model to get the text generated

In [9]:
response = bedrock_runtime.invoke_model(body=req_with_model_params,
                                       modelId='amazon.titan-text-express-v1')
print(response)

{'ResponseMetadata': {'RequestId': '759db37d-a1cd-4bc7-aeb5-143c12747685', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 07 May 2025 04:34:18 GMT', 'content-type': 'application/json', 'content-length': '311', 'connection': 'keep-alive', 'x-amzn-requestid': '759db37d-a1cd-4bc7-aeb5-143c12747685', 'x-amzn-bedrock-invocation-latency': '2264', 'x-amzn-bedrock-output-token-count': '50', 'x-amzn-bedrock-input-token-count': '5'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x10c83e200>}


## Step 6: Extract the generated content from the response body

In [10]:
poem_response = json.loads(response.get('body').read())
poem_response

{'inputTextTokenCount': 5,
 'results': [{'tokenCount': 50,
   'outputText': '\nFrom day to day as I lay down\nI stare at the sky\nFeeling the cold of the breeze touch my skin\nAnd have a deep breath to relax myself\nIts deadly colors brought peace without cease\nAnd a majestic view of the',
   'completionReason': 'LENGTH'}]}

In [12]:
poem = poem_response['results'][0]['outputText']
print(poem)


From day to day as I lay down
I stare at the sky
Feeling the cold of the breeze touch my skin
And have a deep breath to relax myself
Its deadly colors brought peace without cease
And a majestic view of the
